In [4]:
import networkx as nx
import niche
import metrics
import csv
import random
import numpy as np
out_location = "paper_webs"

In [5]:
# intro1_metric - final biomass of first introduced species
# intro2_metric - final biomass of second introduced species
# es_metric - change in ecosystem service amount
# bio_metric - fraction resident species remaining
# es_raw - raw final ecosystem service amount (used in trade-off plots)
metric_header = ["intro1_metric","intro2_metric","es_metric","bio_metric","es_raw"]
management_ops = ["ct","inv1","inv2","invb"]

In [6]:
# Generate final metrics for plotting and save to file 

fif_webs_file = open('./' + out_location + '/over15_webs.txt','r')

# counterfactual metric results (only 1 per web, so all listed in the same file)
ct_file = open('./' + out_location + '/counterfactual_results_newmet.csv','w')
ct_writer = csv.writer(ct_file,dialect="excel")
ct_writer.writerow(["id"] + metric_header)

# results for if the ecosystem service is provided by one basal species
es_res_files = {}
es_res_writers = {}
es_rsd_files = {}
es_rsd_writers = {}
for m in management_ops:
    es_res_files[m] = open('./' + out_location + f'/one_es_res_{m}.csv','w')
    es_res_writers[m] = csv.writer(es_res_files[m])
    if m != "ct":
        es_rsd_files[m] = open('./' + out_location + f'/one_es_res_{m}_rsd.csv','w')
        es_rsd_writers[m] = csv.writer(es_rsd_files[m])
for m in es_res_writers:
    if m != "ct":
        es_res_writers[m].writerow(["web_id","es_node_id","es_change","es_raw"])
    else:
        es_res_writers[m].writerow(["web_id","es_node_id","es_mean_change","es_med_change","es_mean_raw","es_med_raw"])
for m in es_rsd_writers:
    es_rsd_writers[m].writerow(["web_id","one_es_rsd"])

for row in fif_webs_file:
    i = int(row)
    
    # initial biomass vector for web (before introduction, after run to equilibrium)
    webi,_,_,_,b_init_web = niche.read_web_from_file("./" + out_location + "/web_" + str(i) + "_2000")
        
    # save metrics for counterfactual dynamics (4000 total timesteps with no introduction)
    webf,_,_,_,b_final_ct = niche.read_web_from_file("./" + out_location + "/web_" + str(i) + "_2000_2000")
    intro1_ct, intro2_ct, es_ct, bio_ct, es_raw = metrics.get_metrics(webi,webf,b_init_web,b_final_ct,None,None,metrics.es_linear)
    ct_writer.writerow([i, intro1_ct, intro2_ct, es_ct, bio_ct, es_raw])
    one_es_res = []
    one_es_res_raw = []
    for node in niche.get_basal_ids(webi):
        ges = metrics.get_es_spec(b_init_web,b_final_ct,node)
        one_es_res.append(ges[0])
        one_es_res_raw.append(ges[1])
        es_res_writers['ct'].writerow([i,node,ges[0],ges[1]])
            
    # first introduced species
    with open('./' + out_location + '/introduced_1.csv','r') as inv_1_info:
        inv1_reader = csv.reader(inv_1_info)
        next(inv1_reader)
        inv_1_res_dict = {}
        for line in inv1_reader:
            inv_id = int(line[0])
            # invaded web with additional 2000 timesteps after invasion
            webf,_,_,_,b_final_1 = niche.read_web_from_file("./" + out_location + "/web_" + str(i) + "_inv/one/web_" + str(i) + \
                                                         "_inv_" + str(inv_id) + "_2000")
            intro1_inv, intro2_inv, es_inv, bio_inv, es_raw = metrics.get_metrics(webi,webf,b_init_web,b_final_1,inv_id,None,metrics.es_linear)
            inv_1_res_dict[inv_id] = [intro1_inv, intro2_inv, es_inv, bio_inv, es_raw]
            for_rsd, es_nodes, es_nodes_raw = metrics.get_metrics_single_es(b_init_web, b_final_1, webi)                      
    with open('./' + out_location + '/web_' + str(i) + '_introduced_1_newmet.csv','w') as inv_1_res:
        inv1_writer = csv.writer(inv_1_res,dialect="excel")
        inv1_writer.writerow(["inv1_id"] + metric_header)
        for key in inv_1_res_dict:
            inv1_writer.writerow([key] + inv_1_res_dict[key])
    for node in niche.get_basal_ids(webi):        
        es_res_writers['inv1'].writerow([i,node,np.mean(es_nodes[node]),np.median(es_nodes[node]),np.mean(es_nodes_raw[node]),np.median(es_nodes_raw[node])])
    es_rsd_writers['inv1'].writerow([i,metrics.get_rsd(for_rsd)])

    # second introduced species
    with open('./' + out_location + '/introduced_2.csv','r') as inv_2_info:
        inv2_reader = csv.reader(inv_2_info)
        next(inv2_reader)
        inv_2_res_dict = {}
        for line in inv2_reader:
            inv_id = int(line[0])
            webf,_,_,_,b_final_2 = niche.read_web_from_file("./" + out_location + "/web_" + str(i) + "_inv/two/web_" + str(i) + \
                                                         "_inv_" + str(inv_id) + "_2000")
            intro1_inv, intro2_inv, es_inv, bio_inv, es_raw = metrics.get_metrics(webi,webf,b_init_web,b_final_2,None,inv_id,metrics.es_linear)
            inv_2_res_dict[inv_id] = [intro1_inv, intro2_inv, es_inv, bio_inv, es_raw]
            for_rsd, es_nodes, es_nodes_raw = metrics.get_metrics_single_es(b_init_web, b_final_2, webi)                         
    with open('./' + out_location + '/web_' + str(i) + '_introduced_2_newmet.csv','w') as inv_2_res:
        inv2_writer = csv.writer(inv_2_res,dialect="excel")
        inv2_writer.writerow(["inv2_id"] + metric_header)
        for key in inv_2_res_dict:
            inv2_writer.writerow([key] + inv_2_res_dict[key])
        inv_2_res.close()
    es_rsd_writers['inv2'].writerow([i,metrics.get_rsd(for_rsd)])
    for node in niche.get_basal_ids(webi):
        es_res_writers['inv2'].writerow([i,node,np.mean(es_nodes[node]),np.median(es_nodes[node]),np.mean(es_nodes_raw[node]),np.median(es_nodes_raw[node])])
    
    # both introduced species
    with open('./' + out_location + '/introduced_both.csv','r') as inv_both_info:
        inv_both_reader = csv.reader(inv_both_info)
        next(inv_both_reader)
        inv_both_res_dict = {}
        for line in inv_both_reader:
            inv1_id = int(line[0])
            inv2_id = int(line[1])
            webf,_,_,_,b_final_b = niche.read_web_from_file("./" + out_location + "/web_" + str(i) + "_inv/both/web_" + str(i) + "_inv_" + str(inv1_id) + '_' + str(inv2_id) + "_2000")
            intro1_inv, intro2_inv, es_inv, bio_inv, es_raw = metrics.get_metrics(webi,webf,b_init_web,b_final_b,inv1_id,inv2_id,metrics.es_linear)
            inv_both_res_dict[(inv1_id,inv2_id)] = [intro1_inv, intro2_inv, es_inv, bio_inv, es_raw]
            for_rsd, es_nodes, es_nodes_raw = metrics.get_metrics_single_es(b_init_web, b_final_b, webi)    
    with open('./' + out_location + '/web_' + str(i) + '_introduced_both_newmet.csv','w') as inv_both_res:
        inv_both_writer = csv.writer(inv_both_res,dialect="excel")
        inv_both_writer.writerow(["inv1_id","inv2_id"] + metric_header)
        for key in inv_both_res_dict:
            inv_both_writer.writerow([key[0],key[1]] + inv_both_res_dict[key])
    es_rsd_writers['invb'].writerow([i,metrics.get_rsd(for_rsd)])
    for node in niche.get_basal_ids(webi):
        es_res_writers['invb'].writerow([i,node,np.mean(es_nodes[node]),np.median(es_nodes[node]),np.mean(es_nodes_raw[node]),np.median(es_nodes_raw[node])])

fif_webs_file.close()
ct_file.close()
for m in es_res_files:
    es_res_files[m].close()
for m in es_rsd_files:
    es_rsd_files[m].close()